In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
# from sklearn.preprocessing import StandardScaler
from joblib import dump
from pathlib import Path

In [2]:
file_path = Path("resources/movies.csv")
df_movie = pd.read_csv(file_path)

In [9]:
nan_values = df_movie[df_movie.isnull().any(axis=1)]

print (nan_values)

                 name rating   genre  year                           released  \
16               Fame      R   Drama  1980       May 16, 1980 (United States)   
19         Stir Crazy      R  Comedy  1980  December 12, 1980 (United States)   
24       Urban Cowboy     PG   Drama  1980       June 6, 1980 (United States)   
25     Altered States      R  Horror  1980  December 25, 1980 (United States)   
26    Little Darlings      R  Comedy  1980     March 21, 1980 (United States)   
...               ...    ...     ...   ...                                ...   
7663     More to Life    NaN   Drama  2020   October 23, 2020 (United States)   
7664      Dream Round    NaN  Comedy  2020   February 7, 2020 (United States)   
7665    Saving Mbango    NaN   Drama  2020          April 27, 2020 (Cameroon)   
7666     It's Just Us    NaN   Drama  2020    October 1, 2020 (United States)   
7667        Tee em el    NaN  Horror  2020    August 19, 2020 (United States)   

      score    votes       

In [47]:
df_movie["released"] = df_movie["released"].str.split("\s+\(").str[0]
#df_movie["released_country"] = df_movie["released"].str.split("\s+\(").str[1]
df_movie["released"] = pd.to_datetime(df_movie['released'])


In [48]:
df_movie ["released"] = df_movie["released"].dt.month_name()

In [49]:
df_movie.head()

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
0,The Shining,R,Drama,1980,June,8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0
1,The Blue Lagoon,R,Adventure,1980,July,5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,June,8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0
3,Airplane!,PG,Comedy,1980,July,7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0
4,Caddyshack,R,Comedy,1980,July,7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0


In [50]:
df_movie = df_movie.drop(columns=['score', 'votes', 'year'])

In [51]:
df_movie.head()

,name,rating,genre,released,director,writer,star,country,budget,gross,company,runtime
0,The Shining,R,Drama,June,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0
1,The Blue Lagoon,R,Adventure,July,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,June,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0
3,Airplane!,PG,Comedy,July,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0
4,Caddyshack,R,Comedy,July,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0


In [52]:
df_movie['trixbudget']= df_movie['gross'] - 2*df_movie['budget']

In [53]:
df_movie = df_movie.dropna()

In [54]:
df_movie.head()

,name,rating,genre,released,director,writer,star,country,budget,gross,company,runtime,trixbudget
0,The Shining,R,Drama,June,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0,8998772.0
1,The Blue Lagoon,R,Adventure,July,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0,49853106.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,June,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0,502375067.0
3,Airplane!,PG,Comedy,July,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0,76453539.0
4,Caddyshack,R,Comedy,July,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0,27846344.0


In [55]:
df_movie['success'] = df_movie['trixbudget'].map(lambda x: x>0).astype(int)

In [56]:
df_movie_clean = df_movie.drop(columns=['name', 'trixbudget', 'gross'])

In [57]:
df_movie_clean.head()

,rating,genre,released,director,writer,star,country,budget,company,runtime,success
0,R,Drama,June,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,Warner Bros.,146.0,1
1,R,Adventure,July,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,Columbia Pictures,104.0,1
2,PG,Action,June,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,Lucasfilm,124.0,1
3,PG,Comedy,July,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,Paramount Pictures,88.0,1
4,R,Comedy,July,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,Orion Pictures,98.0,1


In [58]:
df_movie['success'] = df_movie['trixbudget'].map(lambda x: x>0).astype(int)

In [59]:
df_movie.sort_values(by="gross", ascending=True)

,name,rating,genre,released,director,writer,star,country,budget,gross,company,runtime,trixbudget,success
3203,Trojan War,PG-13,Comedy,October,George Huang,Andy Burg,Will Friedle,United States,15000000.0,3.090000e+02,Daybreak,85.0,-2.999969e+07,0
272,Parasite,R,Horror,March,Charles Band,Alan J. Adler,Robert Glaudini,United States,800000.0,2.270000e+03,Embassy Pictures,85.0,-1.597730e+06,0
3681,Ginger Snaps,Not Rated,Drama,May,John Fawcett,Karen Walton,Emily Perkins,Canada,5000000.0,2.554000e+03,Copperheart Entertainment,108.0,-9.997446e+06,0
2434,Philadelphia Experiment II,PG-13,Action,June,Stephen Cornwell,Wallace C. Bennett,Brad Johnson,United States,5000000.0,2.970000e+03,Trimark Pictures,97.0,-9.997030e+06,0
5640,Tanner Hall,R,Drama,January,Francesca Gregorini,Tatiana von Fürstenberg,Rooney Mara,United States,3000000.0,5.073000e+03,Two Prong Lesson,96.0,-5.994927e+06,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7244,Avengers: Infinity War,PG-13,Action,April,Anthony Russo,Christopher Markus,Robert Downey Jr.,United States,321000000.0,2.048360e+09,Marvel Studios,149.0,1.406360e+09,1
6663,Star Wars: Episode VII - The Force Awakens,PG-13,Action,December,J.J. Abrams,Lawrence Kasdan,Daisy Ridley,United States,245000000.0,2.069522e+09,Lucasfilm,138.0,1.579522e+09,1
3045,Titanic,PG-13,Drama,December,James Cameron,James Cameron,Leonardo DiCaprio,United States,200000000.0,2.201647e+09,Twentieth Century Fox,194.0,1.801647e+09,1
7445,Avengers: Endgame,PG-13,Action,April,Anthony Russo,Christopher Markus,Robert Downey Jr.,United States,356000000.0,2.797501e+09,Marvel Studios,181.0,2.085501e+09,1


In [60]:
df_movie_success = df_movie.loc[df_movie['success'] == 1]

df_movie_failed = df_movie.loc[df_movie['success'] == 0]

In [61]:
director_other = df_movie['director'].value_counts() < 10
df_movie.loc[df_movie['director'].isin(director_other.index[director_other]), 'director'] = 'others'

In [62]:
writer_other = df_movie['writer'].value_counts() < 10
df_movie.loc[df_movie['writer'].isin(writer_other.index[writer_other]), 'writer'] = 'others'

In [63]:
star_other = df_movie['star'].value_counts() < 15
df_movie.loc[df_movie['star'].isin(star_other.index[star_other]), 'star'] = 'others'

In [64]:
country_other = df_movie['country'].value_counts() < 50
df_movie.loc[df_movie['country'].isin(country_other.index[country_other]), 'country'] = 'others'

In [65]:
company_other = df_movie['company'].value_counts() < 100
df_movie.loc[df_movie['company'].isin(company_other.index[company_other]), 'company'] = 'others'

In [66]:
df_movie.head()

,name,rating,genre,released,director,writer,star,country,budget,gross,company,runtime,trixbudget,success
0,The Shining,R,Drama,June,others,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0,8998772.0,1
1,The Blue Lagoon,R,Adventure,July,others,others,others,United States,4500000.0,58853106.0,Columbia Pictures,104.0,49853106.0,1
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,June,others,others,others,United States,18000000.0,538375067.0,others,124.0,502375067.0,1
3,Airplane!,PG,Comedy,July,others,others,others,United States,3500000.0,83453539.0,Paramount Pictures,88.0,76453539.0,1
4,Caddyshack,R,Comedy,July,others,others,Chevy Chase,United States,6000000.0,39846344.0,others,98.0,27846344.0,1


In [67]:
df_movie = df_movie.drop(columns=['name', 'trixbudget'])

In [68]:
df_movie_dummies = pd.get_dummies(df_movie)

In [69]:
df_movie_dummies

,budget,gross,runtime,success,rating_Approved,rating_G,rating_NC-17,rating_Not Rated,rating_PG,rating_PG-13,...,country_others,company_Columbia Pictures,company_New Line Cinema,company_Paramount Pictures,company_Touchstone Pictures,company_Twentieth Century Fox,company_Universal Pictures,company_Walt Disney Pictures,company_Warner Bros.,company_others
0,19000000.0,46998772.0,146.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,4500000.0,58853106.0,104.0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,18000000.0,538375067.0,124.0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
3,3500000.0,83453539.0,88.0,1,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
4,6000000.0,39846344.0,98.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7648,90000000.0,426505244.0,124.0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
7649,85000000.0,319715683.0,99.0,1,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
7650,175000000.0,245487753.0,101.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
7651,135000000.0,111105497.0,100.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [75]:
y = df_movie_dummies["success"]
X = df_movie_dummies.drop(columns=["success","gross"])

In [76]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(4065, 211)

In [77]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=1)
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [78]:
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [79]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.4735547355473555
Testing Data Score: 0.47345132743362833


In [125]:
df_movie_dummies.nunique()

budget                            398
gross                            5418
runtime                           127
success                             2
rating_Approved                     2
                                 ... 
company_Twentieth Century Fox       2
company_Universal Pictures          2
company_Walt Disney Pictures        2
company_Warner Bros.                2
company_others                      2
Length: 213, dtype: int64

In [17]:
df_movie_failed.nunique()

name          2829
rating           9
genre           15
year            41
released        12
director      1522
writer        2129
star          1257
country         38
budget         281
gross         2853
company       1022
runtime        113
trixbudget    2854
success          1
dtype: int64

In [18]:
df_movie.groupby(by="director").count().sort_values(by="gross", ascending=False)["gross"].head(20)

director
Woody Allen          33
Clint Eastwood       31
Steven Spielberg     27
Ron Howard           23
Ridley Scott         23
Steven Soderbergh    23
Joel Schumacher      20
Tim Burton           19
Martin Scorsese      19
Barry Levinson       18
Spike Lee            17
Robert Zemeckis      17
Oliver Stone         17
Brian De Palma       17
Directors            16
Rob Reiner           15
Walter Hill          15
Garry Marshall       15
Renny Harlin         15
Wes Craven           15
Name: gross, dtype: int64

In [19]:
df_movie.groupby(by="writer").count().sort_values(by="gross", ascending=False)["gross"].head(100)

writer
Woody Allen        32
Stephen King       29
John Hughes        20
Luc Besson         14
Joel Coen          13
                   ..
Don Rhymer          5
Stephen Sommers     5
Steve Kloves        5
Diablo Cody         5
Wes Anderson        5
Name: gross, Length: 100, dtype: int64

In [20]:
df_movie.groupby(by="star").count().sort_values(by="gross", ascending=False)["gross"].head(100)

star
Nicolas Cage            40
Tom Hanks               38
Robert De Niro          38
Bruce Willis            34
Tom Cruise              33
                        ..
Cate Blanchett          11
Patrick Swayze          11
Burt Reynolds           11
Michael Caine           11
Joseph Gordon-Levitt    11
Name: gross, Length: 100, dtype: int64

In [21]:
df_movie_failed.groupby(by="company").count().sort_values(by="gross", ascending=False)["gross"].head(30)

company
Warner Bros.                               155
Universal Pictures                         144
Columbia Pictures                          140
Paramount Pictures                         128
Twentieth Century Fox                       74
New Line Cinema                             63
Metro-Goldwyn-Mayer (MGM)                   62
Touchstone Pictures                         49
TriStar Pictures                            45
Miramax                                     43
Walt Disney Pictures                        35
Castle Rock Entertainment                   32
Orion Pictures                              28
Dreamworks Pictures                         27
Summit Entertainment                        25
Fox 2000 Pictures                           24
Morgan Creek Entertainment                  24
Dimension Films                             22
Hollywood Pictures                          22
New Regency Productions                     21
Screen Gems                                 20
Lions

In [43]:
y = df_movie_dummies["y"]


NameError: name 'df_movie_dummies' is not defined